# Getting Started with Amazon Nova Reel Video Generator

This notebook will help you quickly start generating your own videos using Amazon Nova Reel.

## Prerequisites

Be sure you've followed the instructions in the [00_initial_setup.ipynb](../00_initial_setup.ipynb) notebook to get things set up.


## Configure credentials and shared parameters

Run the cell below to set the default session configuration for all AWS SDK calls made by the other cells in this notebook. As written, the code will default to using the user credentials you have set as your "default" via the AWS CLI. If you'd like to use different credentials, you can modify the code below to add `aws_access_key_id` and `aws_secret_access_key` arguments to the setup function.


In [ ]:
import boto3
import logging

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

MODEL_ID = "amazon.nova-reel-v1:0"

# Set default region and credentials.
boto3.setup_default_session(
    region_name="us-east-1"
)

logger.info("AWS SDK session defaults have been set.")

## Invoking the model

Generating a video takes some time - approximately 4 minutes to produce a 6 second video. To accomodate this execution time, the Bedrock Runtime introduces a new asynchronous invocation API. Calling `start_async_invoke()` creates a new invocation job. When the job completes, Bedrock automatically saves the generated video to an S3 bucket you specify.

### Text-to-Video

The code below will generate a video using only a text prompt as input. Modify the `s3_destination_bucket` and `video_prompt` variables at the start of the code below and then run the cell to start generating your video.

Below is an example of the type of output to expect:

<div style="width: 100%">
    <p align="center">
        <video alt="example_text_to_video" controls>
            <source src="../videos/example_text_to_video.mp4" type="video/mp4" style="padding: 4px">
        </video>
        <br>
        <em>Output video</em>
        <br>
        <em>Prompt: Closeup of a large seashell in the sand, gentle waves flow around the shell. Camera zoom in.</em>
    </p>
</div>

In [ ]:
import json
import random
import amazon_video_util

"""
IMPORTANT: Modify the S3 destination (s3_destination_bucket) and video prompt (video_prompt) below.
"""

# Specify an S3 bucket for the video output.
s3_destination_bucket = "nova-videos"  # Change this to a unique bucket name.

# Specify your video generation prompt. Phrase your prompt as a summary rather than a command. Maximum 512 characters.
video_prompt = "Closeup of a large seashell in the sand, gentle waves flow around the shell. Camera zoom in."

"""
STOP: You should not have to modify anything below this line.
"""

# Set up the S3 client.
s3_client = boto3.client("s3")

# Create the S3 bucket.
s3_client.create_bucket(Bucket=s3_destination_bucket)

# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client("bedrock-runtime")

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": video_prompt
        },
    "videoGenerationConfig": {
        "durationSeconds": 6, 
        "fps": 24,
        "dimension": "1280x720",
        "seed": random.randint(0, 2147483646),
    },
}

try:
    # Start the asynchronous video generation job.
    invocation = bedrock_runtime.start_async_invoke(
        modelId="amazon.nova-reel-v1:0",
        modelInput=model_input,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{s3_destination_bucket}"}},
    )

    # This will be used by other cells in this notebook.
    invocation_arn = invocation["invocationArn"]

    # Pretty print the response JSON.
    logger.info("\nResponse:")
    logger.info(json.dumps(invocation, indent=2, default=str))

    # Save the invocation details for later reference. Helpful for debugging and reporting feedback.
    amazon_video_util.save_invocation_info(invocation, model_input)

except Exception as e:
    logger.error(e)

## Checking the status of generation jobs

While your video is generating you can check its status in one of two ways.

### Using get_async_invoke()

If you know the invocation ARN of the job you'd like to check, you can use the the `get_async_invoke()` method to check the job's status. Run the cell below to check the status of the last job you initiated above.


In [ ]:
# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client("bedrock-runtime")

response = bedrock_runtime.get_async_invoke(
    invocationArn=invocation_arn
)

status = response["status"]
logger.info(f"Status: {status}")

# Pretty print the JSON response
logger.info("\nFull response:")
logger.info(json.dumps(response, indent=2, default=str))

### Using list_async_invokes()

The following code demonstrates how to check the status of **all** video generation jobs in your account using the `list_async_invokes()` method. This method is useful when you don't know the invocation ARN of a job you want to check the status of or if you want to create a dashboard that lists running, completed, or failed jobs. This method supports a number of arguments that allow you to filter by status, date, and a `maxResults` limit. It also supports pagination. We don't show the full API in this example.

In [ ]:
# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client("bedrock-runtime")

invocation = bedrock_runtime.list_async_invokes(
    maxResults=10,  # (Optional)
    statusEquals="InProgress",  # (Optional) Can be "Completed", "InProgress", or "Failed". Omit this argument to list all jobs, regardless of status.
    # Note: There are other supported arguments not demonstrated here.
)

# Pretty print the JSON response
logger.info("Invocation Jobs:")
logger.info(json.dumps(invocation, indent=2, default=str))

## Making things easy

We've provided a set of utility functions in the `amazon_video_util.py` script. One of these functions provides a solution to automatically downloading previously completed jobs and monitoring in-progress jobs. Update the `duration_hours` variable below and then run the cell to download and monitor any video generation jobs started in your account during that time.


In [ ]:
from datetime import datetime, timedelta, timezone

# Download and monitor videos from the past N hours.
duration_hours = 1

# Video timestamps are in utc, so we use utc now
from_submit_time = datetime.now(timezone.utc) - timedelta(hours=duration_hours)
amazon_video_util.monitor_and_download_videos("output", submit_time_after=from_submit_time)

Alternatively if you only want to monitor and download a single job you can use the below utility function and pass in the `invocation_arn` of the job to monitor and download. This function will return the local file path for the generated video for successfully completed jobs.

In [ ]:
local_file_path = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

## Viewing output video
You can then view your output video by running the below cell:

In [ ]:
from IPython.display import Video
Video(local_file_path)